In [1]:
import glob
import os
import shutil
import subprocess
import pandas as pd
import time
import numpy as np
import seaborn as sns
from pathlib import Path
import matplotlib
%matplotlib inline
from config import * #config file with path to input and output folders

In [ ]:
#defining function to run ariba on different databases and species
def aribafn(database, species):
    databasepath = workpath.joinpath("ARIBA", database) #create path for database
    os.makedirs(databasepath, exist_ok = True)
    subprocess.run(["ariba", "getref", database, os.path.join(databasepath, "out."+database)]) #downloading database
    
    #preparing database
    subprocess.run(["ariba", "prepareref", "-f", os.path.join(databasepath, "out."+database+".fa"), "-m", os.path.join(databasepath, "out."+database+".tsv"), os.path.join(databasepath, database+".prepareref")])
    
    #reading the list of isolate names
    namelist = sharepath.joinpath(species+"_isolates", species+"_names.txt")
    table = pd.read_table(namelist, names=["Isolate", "Genus", "Species", "Newname"], index_col="Isolate")
    prepareref = workpath.joinpath("ARIBA",database, database+".prepareref")
    outfile=workpath.joinpath("ARIBA", species+"_ARIBA", database) # create path for outfile
    os.makedirs(outfile, exist_ok = True)
    
    #assigning reads and running ariba
    reads=readpath.joinpath("reads", species+"_reads")
    r1_files=glob.glob(os.path.join(reads, "*_R1*" ))
    count=0
    for r1 in r1_files:
        print(time.ctime())
        start=time.time()
        count+=1
        r2=r1.replace("R1", "R2")
        try:
            labname=table["Newname"][r1.split("/")[5].split("_")[0]]
            #labname=table["Newname"][r1.split("/")[5].split("_R")[0]]
            print(count, "finding resistance genes of", labname)
            subprocess.run(["ariba", "run", prepareref, r1, r2, os.path.join(outfile,labname)])
        except:
            print (r1, "does not exist")
        else:
            labname=table["Newname"][r1.split("/")[5].split("_")[0]]
            #labname=table["Newname"][r1.split("/")[5].split("_R")[0]]
            print(count, "finding resistance genes of", labname)
            subprocess.run(["ariba", "run", prepareref, r1, r2, os.path.join(outfile,labname)])            
        end=time.time()
        print((end-start)/60,"mins")
        
            
    # creating summary files for ARIBA output
    wdir = outfile
    os.chdir(wdir)
    !ariba summary out.summary */report.tsv
    os.chdir(homepath)
    
    #removing report.tsv and changing yes and no to 1 and 0 in ariba summary output csv
    a_s= pd.read_csv(os.path.join(outfile, "out.summary.csv"))
    a_s["name"]=a_s["name"].str.split("/").str[0]
    a_s=a_s.replace("yes", "1")
    a_s=a_s.replace("no", "0")
    a_s = a_s[a_s.columns.drop(list(a_s.filter(regex='colour')))]
    a_s.to_csv(os.path.join(outfile, "out."+species+"_"+database+".phandango.csv"), index=False)


    #removing report.tsv from ariba summary output tree
    tree=os.path.join(outfile, "out.summary.phandango.tre")
    new_tree = os.path.join(outfile, "out."+species+"_"+database+".phandango.tre")
    with open(tree) as f:
        data=f.read()
    
    with open(new_tree, "w") as f:
        f.write(data.replace("/report.tsv", ""))

In [ ]:
#downloading and preparing mlst reference database
workpath_human=os.path.join(workpath, "ARIBA_human_mlst")
home="/home/jabin/research/notebooks/"
os.mkdir(workpath_human)
os.chdir(workpath_human)
!ariba pubmlstget "Staphylococcus aureus" get_mlst --verbose
os.chdir(home)

In [ ]:
# running ARIBA_mlst on bovine isolates
sbh_mlst=pd.read_csv(os.path.join(paperpath, "data", "sbhmlst.csv"), index_col="Assembly")
bovine_mlst=os.path.join(workpath, "ARIBA_mlst")
prepareref=os.path.join(bovine_mlst, "get_mlst", "ref_db")
outfile=os.path.join(bovine_mlst, "bovine_mlst_out")

reads=readpath.joinpath("reads", "Bovine_processed")
r1_files=glob.glob(os.path.join(reads, "*_R1*" ))
count=0
for r1 in r1_files:
    print(time.ctime())
    start=time.time()
    count+=1
    r2=r1.replace("R1", "R2")
    labname=sbh_mlst["newname"][r1.split("/")[5].split("_")[1]]
    print(count, "finding resistance genes of", labname)
    subprocess.run(["ariba", "run", prepareref, r1, r2, os.path.join(outfile,labname)])           
    end=time.time()
    print((end-start)/60,"mins")

In [ ]:
# running ARIBA_mlst on human isolates
sbh_mlst=pd.read_csv(os.path.join(paperpath, "data", "sbhmlst.csv"), index_col="Assembly")
human_mlst=os.path.join(workpath, "ARIBA_human_mlst")
prepareref=os.path.join(human_mlst, "get_mlst", "ref_db")
outfile=os.path.join(human_mlst, "human_mlst_out")

reads=readpath.joinpath("reads", "Human_processed")
r1_files=glob.glob(os.path.join(reads, "*_R1*" ))
count=0
for r1 in r1_files:
    print(time.ctime())
    start=time.time()
    count+=1
    r2=r1.replace("R1", "R2")
    labname=sbh_mlst["newname"][r1.split("/")[5].split("_")[1]]
    print(count, "finding resistance genes of", labname)
    subprocess.run(["ariba", "run", prepareref, r1, r2, os.path.join(outfile,labname)])           
    end=time.time()
    print((end-start)/60,"mins")

In [ ]:
#removing report.tsv and changing yes and no to 1 and 0 in ariba summary output csv
outfile=os.path.join(readpath, "Comparative", "ARIBA")
a_s= pd.read_csv(os.path.join(outfile, "out_vfdb_full.phandango.csv"))
a_s["name"]=a_s["name"].str.split("/").str[2]
a_s=a_s.replace("yes", "1")
a_s=a_s.replace("no", "0")
a_s = a_s[a_s.columns.drop(list(a_s.filter(regex='colour')))]
a_s.to_csv(os.path.join(outfile, "out_all_vfdb_full.phandango.csv"), index=False)


#removing report.tsv from ariba summary output tree
tree=os.path.join(outfile, "out_vfdb_full.phandango.tre")
new_tree = os.path.join(outfile, "out_all_vfdb_full.phandango.tre")
with open(tree) as f:
    data=f.read()

with open(new_tree, "w") as f:
    f.write(data.replace("/report.tsv", ""))

In [ ]:
# creating summary files for ARIBA output
outfile=os.path.join(readpath, "Comparative", "ARIBA")
wdir = outfile
os.chdir(wdir)
!ariba summary out_vfdb_full *_ARIBA/vfdb_full/*/report.tsv
os.chdir(homepath)

a_s= pd.read_csv(os.path.join(outfile, "out_vfdb_full.phandango.csv"))
a_s["name"]=a_s["name"].str.split("/").str[2]
a_s=a_s.replace("yes", "1")
a_s=a_s.replace("no", "0")
a_s = a_s[a_s.columns.drop(list(a_s.filter(regex='colour')))]
a_s.to_csv(os.path.join(outfile, "out_all_vfdb_full.phandango.csv"), index=False)


#removing report.tsv from ariba summary output tree
tree=os.path.join(outfile, "out_vfdb_full.phandango.tre")
new_tree = os.path.join(outfile, "out_all_vfdb_full.phandango.tre")
with open(tree) as f:
    data=f.read()

with open(new_tree, "w") as f:
    f.write(data.replace("/report.tsv", ""))

In [ ]:
# creating dataset for itol tree
def itree(database, species, colour)
outfile=os.path.join(readpath, "Comparative", "ARIBA")
inputpath= os.path.join(outfile, species+'_ARIBA', database)
export_location = os.path.join(outfile, species+"_ARIBA", database, species+database+"_tree.pdf")
oldtree = os.path.join(inputpath, 'out.'+species+"_"+database+".phandango.tre")
tree = os.path.join(inputpath, 'out.'+species+"_"+database+".phandango.tree")
shutil.copy(oldtree, tree)
csv_phandango = os.path.join(inputpath, "out."+species+"_"+database+".phandango.csv")
dataset = os.path.join(inputpath, 'binary_'+species+database+".txt")
import csv
field = list(pd.read_csv(csv_phandango, nrows=0))
field_shapes = ",".join(["1"] * len(field[1:]))
field_labels = ",".join(field[1:])
output=open(dataset, "w")
output.writelines("DATASET_BINARY"+"\n"+"\n"+"SEPARATOR COMMA"+"\n"+"\n"
                  +"DATASET_LABEL"+","+database+"_"+species+"\n"+"\n"+"#Dataset color"
                  +"\n"+"COLOR"+","+color+"\n"+"\n"+"FIELD_SHAPES"+","+field_shapes
                  +"\n"+"\n"+"FIELD_LABELS"+","+field_labels+"\n"+"\n"+"SHOW_LABELS"+","
                  +"1"+"\n"+"\n"+"DATA"+"\n"+"\n")

with open(csv_phandango, "r") as f:
    f_csv = csv.reader(f) 
    headers = next(f_csv) 
    with open (dataset, "a") as output:
        for row in f_csv:
            output.write(",".join(row)+"\n")


In [2]:
#extracting genes from fasta file and renaming the header to the isolate name using seqkit
def extract(database, gene):
    mlstdata=pd.read_csv(os.path.join(paperpath, "data", "sbhmlst.csv"), index_col="old_name")
    filename=glob.glob(os.path.join(readpath, "Comparative", "ARIBA", "*", database, "*", "assemblies.fa"))
    homepath="/home/jabin/Documents/Bovine_isolates/"
    for n in filename:
        labname=mlstdata["newname"][n.split("/")[7]]
        with open (os.path.join(homepath, "individual_genes", database+"_"+gene+"_genes.fasta"), "a") as f:
            ex=subprocess.Popen(["seqkit", "grep", "-r", "-p", "^"+gene, n], stdout=subprocess.PIPE)
            subprocess.Popen(["seqkit","replace", "-p",".+", "-r", labname],stdin=ex.stdout, stdout=f)

        
    f.close()

In [3]:
extract("vfdb_full", "vWbp")

In [ ]:
#defining function to run ariba summary on different databases and species
def aribafn(database, species):
    
    #reading the list of isolate names
    outfile=readpath.joinpath("Comparative","ARIBA", species+"_ARIBA", database) # create path for outfile
    os.makedirs(outfile, exist_ok = True)
                
    # creating summary files for ARIBA output
    wdir = outfile
    os.chdir(wdir)
    !ariba summary out.summary */report.tsv
    os.chdir(homepath)
    
    #removing report.tsv and changing yes and no to 1 and 0 in ariba summary output csv
    a_s= pd.read_csv(os.path.join(outfile, "out.summary.csv"))
    a_s["name"]=a_s["name"].str.split("/").str[0]
    a_s=a_s.replace("yes", "1")
    a_s=a_s.replace("no", "0")
    a_s = a_s[a_s.columns.drop(list(a_s.filter(regex='colour')))]
    a_s.to_csv(os.path.join(outfile, "out."+species+"_"+database+".phandango.csv"), index=False)


    #removing report.tsv from ariba summary output tree
    tree=os.path.join(outfile, "out.summary.phandango.tre")
    new_tree = os.path.join(outfile, "out."+species+"_"+database+".phandango.tre")
    with open(tree) as f:
        data=f.read()
    
    with open(new_tree, "w") as f:
        f.write(data.replace("/report.tsv", ""))

In [ ]:
aribafn("vfdb_core", "Bovine")

In [ ]:
#Visualizing AST results using seaborn distplot 
def ast (antibiotic, color):
    #input and output files:change path and filename accordingly
    AST =  pd.read_excel(os.path.join(sharepath, "ARIBA", "ARIBA_csv", "AST test results data for graphs_onefarm.xlsx"))
    outfile = os.path.join(sharepath, "ARIBA", "ARIBA_csv", drug1+"_density_AST_onefarm.png")  
    
    distfig=sns.distplot(AST[antibiotic], color=color)
    distfig.set_title("Antibiotic sensitivity test density distribution of "+ antibiotic)
    distfig.set_xlabel(antibiotic+" diameter(in mm)")
    distfig.set_ylabel('density')
    distfig.figure.savefig(outfile, dpi=200)